# Imports

In [22]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pickle

account="account1"
dataName="sampleData1"

## Read Data & Clean

In [23]:
# reading csv file  
rawDF=pd.read_csv("../delete/SampleCrimeData.csv") 

#This is specific to this use case. Modify it as per need
rawDF.replace(',','', regex=True, inplace=True)

## Preview Data

In [24]:
rawDF.head()

,Year,Population1,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate
0,1997,267783607,1636096,611.0,498534,186.2,1023201,382.1,11558475,4316.30,2460526,918.8,7743760,2891.80,1354189,505.7
1,1998,270248003,1533887,567.6,447186,165.5,976583,361.4,10951827,4052.50,2332735,863.2,7376311,2729.50,1242781,459.9
2,1999,272690813,1426044,523.0,409371,150.1,911740,334.3,10208334,3743.60,2100739,770.4,6955520,2550.70,1152075,422.5
3,2000,281421906,1425486,506.5,408016,145.0,911706,324.0,10182584,3618.30,2050992,728.8,6971590,2477.30,1160002,412.2
4,20015,285317559,1439480,504.5,423557,148.5,909023,318.6,10437189,3658.10,2116531,741.8,7092267,2485.70,1228391,430.5


## Detect Data Types 

In [25]:
cols = rawDF.columns
num_cols = rawDF._get_numeric_data().columns
#Categorical variables
list(set(cols) - set(num_cols))

['Robbery',
 'Burglary',
 'Larceny-\r\ntheft rate',
 'Population1',
 'Property \r\ncrime',
 'Larceny-\r\ntheft',
 'Violent\r\ncrime2',
 'Aggravated \r\nassault',
 'Property \r\ncrime \r\nrate',
 'Motor \r\nvehicle \r\ntheft']

## Provide the categorical column names 

In [26]:
categorical_cols=['Year','Population1']  ## Data Sci/Eng Input
string = 'Code'
code_cols = [x + string for x in categorical_cols]
print(code_cols);
#code_cols=['YearCode','PopulationCode']

rawDF[categorical_cols]=rawDF[categorical_cols].apply(lambda x: x.astype('category'))
#rawDF.dtypes

['YearCode', 'Population1Code']


In [27]:
# rawDF.select_dtypes(include='category') #Verify the result

## Extract the categorical keys and modify  

In [28]:
rawDF[code_cols]=rawDF[categorical_cols].apply(lambda x: x.cat.codes)
rawDF.dtypes
code_cols.extend(categorical_cols)
print(code_cols)
catDFDataMap=rawDF[code_cols]

rawDF=rawDF.drop(categorical_cols, axis = 1) 

['YearCode', 'Population1Code', 'Year', 'Population1']


## Extract Keys 

In [29]:
originalColumns=rawDF.columns

def minMax(x):
    return pd.Series(index=['min','max'],data=[x.min(),x.max()])

originalMinMax=rawDF.apply(minMax)


### Preview Keys

In [30]:
print(type(originalColumns))
print(originalColumns.values.tolist())
originalMinMax

<class 'pandas.core.indexes.base.Index'>
['Violent\r\ncrime2', 'Violent \r\ncrime \r\nrate', 'Robbery', 'Robbery \r\nrate', 'Aggravated \r\nassault', 'Aggravated \r\nassault rate', 'Property \r\ncrime', 'Property \r\ncrime \r\nrate', 'Burglary', 'Burglary \r\nrate', 'Larceny-\r\ntheft', 'Larceny-\r\ntheft rate', 'Motor \r\nvehicle \r\ntheft', 'Motor \r\nvehicle \r\ntheft \r\nrate', 'YearCode', 'Population1Code']


,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate,YearCode,Population1Code
min,1153022,361.6,322905,101.3,1023201,229.2,10019601,2450.70,1515096,468.9,5638455,1745.00,1100472,215.4,0,0
max,1636096,611.0,498534,186.2,976583,382.1,9882212,4316.30,2460526,918.8,7743760,2891.80,959059,505.7,19,19


## Scale the columns 

In [31]:
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(rawDF), columns=rawDF.columns)

## Preview result

In [32]:
df_scaled.head()

,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate,YearCode,Population1Code
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
1,0.788420,0.825982,0.707634,0.756184,0.842732,0.864617,0.833313,0.858598,0.864833,0.876417,0.825465,0.858476,0.833068,0.842232,0.052632,0.052632
2,0.565176,0.647153,0.492322,0.574794,0.623981,0.687377,0.629025,0.693021,0.619446,0.670149,0.625593,0.702564,0.697156,0.713400,0.105263,0.105263
3,0.564021,0.580994,0.484607,0.514723,0.623866,0.620013,0.621950,0.625858,0.566828,0.577684,0.633227,0.638559,0.709033,0.677919,0.157895,0.157895
4,0.592990,0.572975,0.573094,0.555948,0.614815,0.584696,0.691907,0.647191,0.636150,0.606579,0.690547,0.645884,0.811506,0.740958,0.947368,0.210526


## Replace Column names 

In [33]:
#df_scaled.columns = [''] * len(df_scaled.columns)
col_list = ['dx' + str(x) for x in range(1,1+len(df_scaled.columns))]
df_scaled.columns = col_list

columnMapping = pd.DataFrame([col_list],columns=originalColumns) 

columnMapping.head()

,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate,YearCode,Population1Code
0,dx1,dx2,dx3,dx4,dx5,dx6,dx7,dx8,dx9,dx10,dx11,dx12,dx13,dx14,dx15,dx16


# Save Data and Keys

In [37]:
df_scaled.to_csv('../Bank/'+account+'/'+dataName+'.csv', index=False)
df_scaled.to_json('../Bank/'+account+'/'+dataName+'.json', orient='columns')
originalMinMax.to_csv('../AccountKeys/'+account+'_'+dataName+'-MiMa.csv', index=False)
originalMinMax.to_json('../AccountKeys/'+account+'_'+dataName+'-MiMa.json',orient='columns') # orient= columns  split
columnMapping.to_json('../AccountKeys/'+account+'_'+dataName+'-CoMap.json',orient='records') # orient= columns  split records
#Categorical Data
catDFDataMap.to_csv('../AccountKeys/'+account+'_'+dataName+'-CatCode.csv', index=False)
catDFDataMap.to_json('../AccountKeys/'+account+'_'+dataName+'-CatCode.json',orient='columns') # orient= columns  split
catDFDataMap.to_json('../AccountKeys/'+account+'_'+dataName+'-CatCodeMap.json',orient='records') # orient= columns  split records


with open('../AccountKeys/'+account+'_'+dataName+'.txt', "wb") as fp:   #Pickling
    pickle.dump(originalColumns.values.tolist(), fp)

## Test saved data

In [40]:
df_scaledRead=pd.read_csv('../Bank/'+account+'/'+dataName+'.csv') #read data
originalMinMaxRead=pd.read_csv('../AccountKeys/'+account+'_'+dataName+'-MiMa.csv') #Read column mapping
catDFDataMapRead =pd.read_csv('../AccountKeys/'+account+'_'+dataName+'-CatCode.csv') #Read Categorical data

with open('../AccountKeys/'+account+'_'+dataName+'.txt', "rb") as fp:   # Unpickling
     originalColumnsRead = pickle.load(fp)
df_scaledRead.columns = originalColumnsRead

pd.concat([df_scaledRead, catDFDataMapRead.reindex(df_scaledRead.index)], axis=1)


,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate,YearCode,Population1Code,YearCode,Population1Code,Year,Population1
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0,0,1997,267783607
1,0.788420,0.825982,0.707634,0.756184,0.842732,0.864617,0.833313,0.858598,0.864833,0.876417,0.825465,0.858476,0.833068,0.842232,0.052632,0.052632,1,1,1998,270248003
2,0.565176,0.647153,0.492322,0.574794,0.623981,0.687377,0.629025,0.693021,0.619446,0.670149,0.625593,0.702564,0.697156,0.713400,0.105263,0.105263,2,2,1999,272690813
3,0.564021,0.580994,0.484607,0.514723,0.623866,0.620013,0.621950,0.625858,0.566828,0.577684,0.633227,0.638559,0.709033,0.677919,0.157895,0.157895,3,3,2000,281421906
4,0.592990,0.572975,0.573094,0.555948,0.614815,0.584696,0.691907,0.647191,0.636150,0.606579,0.690547,0.645884,0.811506,0.740958,0.947368,0.210526,18,4,20015,285317559
5,0.560276,0.532478,0.557431,0.527680,0.555387,0.525180,0.696877,0.632451,0.672875,0.618137,0.673976,0.615364,0.838859,0.749225,0.210526,0.263158,4,5,2002,287973924
6,0.477471,0.457899,0.520017,0.485277,0.446162,0.432963,0.693466,0.611331,0.676664,0.604801,0.659452,0.585542,0.860706,0.751981,0.263158,0.315789,5,6,2003,290788976
7,0.428642,0.407378,0.447335,0.416961,0.406863,0.388489,0.659539,0.570004,0.665676,0.581018,0.616839,0.538280,0.825681,0.709955,0.315789,0.368421,6,7,2004,293656842
8,0.492105,0.430634,0.538254,0.465253,0.456923,0.402878,0.619798,0.525729,0.677313,0.573461,0.543860,0.473317,0.822696,0.693765,0.368421,0.421053,7,8,2005,296507061
9,0.583971,0.471933,0.719363,0.573616,0.496987,0.410726,0.577167,0.480221,0.719140,0.587242,0.469247,0.408266,0.766336,0.636583,0.421053,0.473684,8,9,2006,299398484


In [39]:
df_scaledRead.head()

,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate,YearCode,Population1Code
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
1,0.788420,0.825982,0.707634,0.756184,0.842732,0.864617,0.833313,0.858598,0.864833,0.876417,0.825465,0.858476,0.833068,0.842232,0.052632,0.052632
2,0.565176,0.647153,0.492322,0.574794,0.623981,0.687377,0.629025,0.693021,0.619446,0.670149,0.625593,0.702564,0.697156,0.713400,0.105263,0.105263
3,0.564021,0.580994,0.484607,0.514723,0.623866,0.620013,0.621950,0.625858,0.566828,0.577684,0.633227,0.638559,0.709033,0.677919,0.157895,0.157895
4,0.592990,0.572975,0.573094,0.555948,0.614815,0.584696,0.691907,0.647191,0.636150,0.606579,0.690547,0.645884,0.811506,0.740958,0.947368,0.210526


# Preffered way to share data

Step 1: Gor to "Bank" folder/directory  
Step 2: Check if the account01/02... folder is created and has the data  
Step 3: If you want it to publish it for public usage, then submit to Git repository, and raise a merge request  
Step 4: For private usage you can transmit the data any way you feel comfortable with,  
Step 5: Do not commit data in "AccountKeys" unless you intend to make data public  
Step 6: Delete data from "delete" folder, before commiting/pushing data to git  